# Training Models with the Data-Driven Library

The datadriven library provides an extensible command-line interface for training, evaluating, and predicting data-driven simulators. However, you may prefer training and sweeping models inside a notebook. This notebook provides an example for doing so.

## Set Working Directory and Import Necessary Libraries

In [ ]:
cd ..

In [ ]:
from hydra.experimental import initialize, compose
from omegaconf import DictConfig, ListConfig, OmegaConf
from model_loader import available_models
from base import plot_parallel_coords
import logging
import matplotlib.pyplot as plt
import numpy as np
from rich import print
from rich.logging import RichHandler
import copy
import pandas as pd

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler()]
)
logger = logging.getLogger("ddm_training")
logger.setLevel(logging.INFO)

## Initialize Configuration

While you can provide every argument manually, there is benefit in directly using the `hydra` config class to load an existing configuration file. This way you can ensure your parameters are saved to a file for later use, and you automatically gain the benefit of all the logging and model artifacts that are provided by our workflow of `hydra` and `mlflow`.

If you want to override any settings of the configurations, provide them in a list of `overrides` as shown below.

In [ ]:
initialize(config_path="../conf", job_name="ddm_training")
cfg = compose(config_name="config", overrides=["data=cartpole_st1_at", "model=torch"])

In [ ]:
print(OmegaConf.to_yaml(cfg))

In [ ]:
# Extract features from yaml file
input_cols = cfg['data']['inputs']
output_cols = cfg['data']['outputs']
augmented_cols = cfg['data']['augmented_cols']
dataset_path = cfg['data']['path']
iteration_order = cfg['data']['iteration_order']
episode_col = cfg['data']['episode_col']
iteration_col = cfg['data']['iteration_col']
max_rows = cfg['data']['max_rows']

## Model Trainer

To make it easy to sweep over models later, we create a simple `train_models` function here:

In [ ]:
def train_models(config=cfg):

    logger.info(f'Model type: {available_models[config["model"]["name"]]}')
    Model = available_models[config["model"]["name"]]
    global model
    model = Model()
    logger.info(f"Loading data from {dataset_path}")
    global X, y
    X, y = model.load_csv(
        input_cols=input_cols,
        output_cols=output_cols,
        augm_cols=list(augmented_cols),
        dataset_path=dataset_path,
        iteration_order=iteration_order,
        episode_col=episode_col,
        iteration_col=iteration_col,
        max_rows=max_rows,
    )
    logger.info(f"Building model with parameters: {config}")
    model.build_model(
        **config["model"]["build_params"]
    )


    logger.info(f"Fitting model...")
    model.fit(X, y)
    logger.info(f"Model trained!")

    return model

In [ ]:
model = train_models(cfg)

## Hyperparameter Sweeping

The `datadrivenmodel` has an automatic solution for hyperparameter sweeping and tuning. These settings are provided in the config `model.sweep` parameters. Provide the limits of the variables you want to sweep over and the `sweep` method will automatically parallelize the sweep over the available number of cores and find the optimal solution according to your `scoring_func`.

### Configuration Parameters

You can select the search algorithm you'd like to use: `bayesian` runs bayesian optimiziation (using scikit-optimize), `hyperopt` runs [Tree-Parzen Estimators](https://papers.nips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf) with the `hyperopt` package, `bohb` uses Bayesian Opt/HyperBand, or `optuna` which also runs Tree-Parzen estimators but using the [`optuna`](https://optuna.readthedocs.io/en/stable/) package.

In [ ]:
print(OmegaConf.to_yaml(cfg["model"]["sweep"]))

In [ ]:
params = OmegaConf.to_container(cfg["model"]["sweep"]["params"])
logger.info(f"Sweeping with parameters: {params}")

sweep_df = model.sweep(
    params=params,
    X=X,
    y=y,
    search_algorithm=cfg["model"]["sweep"]["search_algorithm"],
    num_trials=cfg["model"]["sweep"]["num_trials"],
    scoring_func=cfg["model"]["sweep"]["scoring_func"],
    results_csv_path=cfg["model"]["sweep"]["results_csv_path"],
)    

In [ ]:
sweep_df.head()

## Results and Outputs

All outputs are saved to a timestamped directory in `outputs`. This includes the model artifacts, hyperparameter tuning results, and a verbose log of the entire run.

In [ ]:
ls -lh outputs/

In [ ]:
!tree -L 3 outputs/2021-04-21/

### Visualizing Hyperparameter Results

In [ ]:
plot_parallel_coords(sweep_df)

### Reading Saved Runs from CSV

Runs are automatically saved to a CSV in the outputs directory:

In [ ]:
sweep_df2 = pd.read_csv("outputs/2021-04-21/10-29-25/xgboost_gridsearch/search_results.csv")

In [ ]:
plot_parallel_coords(sweep_df2)